In [ ]:
def Gen_electron_Beam(input_filename, sample_electrons):
    
    
    
    with open( input_filename, 'r' ) as stream:
        input_dict = yaml.load(stream, Loader=yaml.SafeLoader)
    
    
    
    
    
    
    # extract beam parameters
    gamma0       = float( input_dict['beam']['gamma'] )
    energyspread = float( input_dict['beam']['energyspread'] )
    emittance    = float( input_dict['beam']['emittance'] )
    beam_size_X    = float( input_dict['beam']['sigmaX'] )        # transverse beam size x axis in microns
    beam_size_Y    = float( input_dict['beam']['sigmaY'] )        # transverse beam size y axis in microns
    beam_length  = float( input_dict['beam']['sigmaL'] )        # longitudinal beam size in microns 
    beam_charge  = float( input_dict['beam']['beam_charge'])
    
    
    
    
    
    
    # rms angles for x and y space
    rms_angle_X   = emittance / beam_size_X / gamma0
    
    rms_angle_Y   = emittance / beam_size_Y / gamma0
    
    
    
    
    
    
    
    
    
    #Mean and Covariant matrices for bivariate gaussian
    
    mean_x = [0,0]  # x-offset and x' offset for focus 
    cov_x = Covariant_Matrix(beam_size_X,rms_angle_X)

    mean_y = [0,0]  # y-offset and y' offset for focus
    cov_y = Covariant_Matrix(beam_size_Y,rms_angle_Y)

    
    
    
    
    
    
    
    
    
    
    #Sampling (x,x') and (y,y')
    x,theta_x = np.random.multivariate_normal(mean_x, cov_x, sample_electrons).T
    y,theta_y = np.random.multivariate_normal(mean_y, cov_y, sample_electrons).T
    
    
    
    gamma        = np.random.normal( gamma0 , gamma0*energyspread , sample_electrons )
    beta         = sqrt(1-1./gamma**2)

    r            = np.sqrt( x**2 + y**2 )

    xi_peak      = a0 * exp( -r**2/w0**2 )


    omega        = np.random.uniform(omega_detector[0], omega_detector[1], sample_electrons) 
    theta        = np.random.uniform(theta_detector[0], theta_detector[1], sample_electrons) 
    phi          = np.random.uniform(0,2*pi,sample_electrons)


    pz0          = gamma*beta*cos(theta_x)*cos(theta_y)
    px0          = gamma*beta*sin(theta_x)
    py0          = gamma*beta*sin(theta_y)
    pt0          = sqrt( 1 + px0**2 + py0**2 + pz0**2 )


    
    return(x,theta_x,y,theta_y,gamma,omega,theta,xi_peak,phi,pt0,px0,py0,pz0)